In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib numpy qiskit-basis-constructor

# Introduzione ai gate frazionari

*Stima dell'utilizzo: meno di 30 secondi su un processore Heron r2 (NOTA: Questa è solo una stima. Il vostro tempo di esecuzione può variare.)*
## Background

### Gate frazionari sui QPU IBM
I gate frazionari sono gate quantistici parametrizzati che consentono l'esecuzione diretta di rotazioni ad angoli arbitrari (entro limiti specifici),
eliminando la necessità di decomporli in più gate di base.
Sfruttando le interazioni native tra qubit fisici, gli utenti possono implementare determinati operatori unitari in modo più efficiente sull'hardware.

I QPU IBM Quantum&reg; Heron supportano i seguenti gate frazionari:
- $R_{ZZ}(\theta)$ per $0 < \theta < \pi / 2$
- $R_X(\theta)$ per qualsiasi valore reale $\theta$

Questi gate possono ridurre significativamente sia la profondità che la durata dei circuiti quantistici.
Sono particolarmente vantaggiosi in applicazioni che si basano fortemente su $R_{ZZ}$ e $R_X$,
come la simulazione hamiltoniana, il Quantum Approximate Optimization Algorithm (QAOA) e i metodi del kernel quantistico.
In questo tutorial, ci concentriamo sul kernel quantistico come esempio pratico.
### Limitazioni
I gate frazionari sono attualmente una funzionalità sperimentale e presentano alcuni vincoli:
- $R_{ZZ}$ è limitato ad angoli nell'intervallo $0 < \theta < \pi / 2$.
- L'uso dei gate frazionari non è supportato per [circuiti dinamici](/guides/classical-feedforward-and-control-flow), [Pauli twirling](/guides/error-mitigation-and-suppression-techniques#pauli-twirling), [probabilistic error cancellation](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-cancellation-pec) (PEC), e [zero-noise extrapolation](/guides/error-mitigation-and-suppression-techniques#zero-noise-extrapolation-zne) (ZNE) (utilizzando [probabilistic error amplification](/guides/error-mitigation-and-suppression-techniques#probabilistic-error-amplification-pea) (PEA)).

I gate frazionari richiedono un flusso di lavoro diverso rispetto all'approccio standard.
Questo tutorial spiega come lavorare con i gate frazionari attraverso un'applicazione pratica.

Consultate quanto segue per maggiori dettagli sui gate frazionari.
- [Gate frazionari](/guides/fractional-gates)
- [Quando *non* utilizzare i gate frazionari](/guides/fractional-gates#when-not-to-use)
## Panoramica
Il flusso di lavoro per l'utilizzo dei gate frazionari segue generalmente il flusso di lavoro dei [pattern Qiskit](/guides/intro-to-patterns).
La differenza chiave è che tutti gli angoli RZZ devono soddisfare il vincolo $0 < \theta \leq \pi/2$.
Esistono due approcci per garantire che questa condizione sia soddisfatta.
Questo tutorial si concentra sul secondo approccio e lo raccomanda.

### 1. Generare valori di parametri che soddisfano il vincolo dell'angolo RZZ
Se siete sicuri che tutti gli angoli RZZ rientrino nell'intervallo valido, potete seguire il flusso di lavoro standard dei pattern Qiskit.
In questo caso, inviate semplicemente i valori dei parametri come parte di un PUB. Il flusso di lavoro procede come segue.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from qiskit import QuantumCircuit, generate_preset_pass_manager
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import unitary_overlap
from qiskit_ibm_runtime import QiskitRuntimeService, SamplerV2

Se tentate di inviare un PUB che include un gate RZZ con un angolo al di fuori dell'intervallo valido, incontrerete un messaggio di errore come:
```
'The instruction rzz is supported only for angles in the range [0, pi/2], but an angle (20.0) outside of this range has been requested; via parameter value(s) γ[0]=10.0, substituted in parameter expression 2.0*γ[0].'
```
Per evitare questo errore, dovreste considerare il secondo approccio descritto di seguito.
### 2. Assegnare i valori dei parametri ai circuiti prima della transpilazione
Il pacchetto `qiskit-ibm-runtime` fornisce un pass di transpilazione specializzato chiamato [`FoldRzzAngle`](https://docs.quantum.ibm.com/api/qiskit-ibm-runtime/transpiler-passes-fold-rzz-angle).
Questo pass trasforma i circuiti quantistici in modo che tutti gli angoli RZZ rispettino il vincolo dell'angolo RZZ.
Se fornite il backend a `generate_preset_pass_manager` o `transpile`, Qiskit applica automaticamente `FoldRzzAngle` ai circuiti quantistici.
Questo vi richiede di assegnare i valori dei parametri ai circuiti quantistici prima della transpilazione.
Il flusso di lavoro procede come segue.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=133
)  # backend should be a heron device or later
backend_name = backend.name
backend_c = service.backend(backend_name)  # w/o fractional gates
backend_f = service.backend(
    backend_name, use_fractional_gates=True
)  # w/ fractional gates
print(f"Backend: {backend_name}")
print(f"No fractional gates: {backend_c.basis_gates}")
print(f"With fractional gates: {backend_f.basis_gates}")
if "rzz" not in backend_f.basis_gates:
    print(f"Backend {backend_name} does not support fractional gates")

Backend: ibm_fez
No fractional gates: ['cz', 'id', 'rz', 'sx', 'x']
With fractional gates: ['cz', 'id', 'rx', 'rz', 'rzz', 'sx', 'x']


Notate che questo flusso di lavoro comporta un costo computazionale più elevato rispetto al primo approccio, poiché implica l'assegnazione dei valori dei parametri ai circuiti quantistici e la memorizzazione locale dei circuiti con parametri vincolati.
Inoltre, esiste un problema noto in Qiskit in cui la trasformazione dei gate RZZ può fallire in determinati scenari. Per una soluzione alternativa, fate riferimento alla sezione [Risoluzione dei problemi](#troubleshooting).
Questo tutorial dimostra come utilizzare i gate frazionari tramite il secondo approccio attraverso un esempio ispirato al metodo del kernel quantistico.
Per comprendere meglio dove è probabile che i kernel quantistici siano utili, raccomandiamo di leggere [Liu, Arunachalam & Temme (2021).](https://www.nature.com/articles/s41567-021-01287-z)

Potete anche seguire il tutorial [Quantum kernel training](/tutorials/quantum-kernel-training) e la lezione [Quantum kernels](/learning/courses/quantum-machine-learning/quantum-kernel-methods) nel corso di Quantum machine learning su IBM Quantum Learning.
### Requisiti
Prima di iniziare questo tutorial, assicuratevi di avere installato quanto segue:
- Qiskit SDK v2.0 o successivo, con supporto per la [visualizzazione](https://docs.quantum.ibm.com/api/qiskit/visualization)
- Qiskit Runtime v0.37 o successivo (`pip install qiskit-ibm-runtime`)
- Qiskit Basis Constructor (`pip install qiskit_basis_constructor`)
### Setup

In [3]:
optimization_level = 2
shots = 2000
reps = 3
rng = np.random.default_rng(seed=123)

In [4]:
def my_zz_feature_map(num_qubits: int, reps: int = 1) -> QuantumCircuit:
    x = ParameterVector("x", num_qubits * reps)
    qc = QuantumCircuit(num_qubits)
    qc.h(range(num_qubits))
    for k in range(reps):
        K = k * num_qubits
        for i in range(num_qubits):
            qc.rz(x[i + K], i)
        pairs = [(i, i + 1) for i in range(num_qubits - 1)]
        for i, j in pairs[0::2] + pairs[1::2]:
            qc.rzz((np.pi - x[i + K]) * (np.pi - x[j + K]), i, j)
    return qc


def quantum_kernel(num_qubits: int, reps: int = 1) -> QuantumCircuit:
    qc = my_zz_feature_map(num_qubits, reps=reps)
    inner_product = unitary_overlap(qc, qc, "x", "y", insert_barrier=True)
    inner_product.measure_all()
    return inner_product


def random_parameters(inner_product: QuantumCircuit) -> np.ndarray:
    return np.tile(rng.random(inner_product.num_parameters // 2), 2)


def fidelity(result) -> float:
    ba = result.data.meas
    return ba.get_int_counts().get(0, 0) / ba.num_shots

Quantum kernel circuits and their corresponding parameter values are generated for systems with 4 to 40 qubits, and their fidelities are subsequently evaluated.

In [5]:
qubits = list(range(4, 44, 4))
circuits = [quantum_kernel(i, reps=reps) for i in qubits]
params = [random_parameters(circ) for circ in circuits]

## Flusso di lavoro con i gate frazionari
### Step 1: Mappare gli input classici al problema quantistico
#### Circuito del kernel quantistico
In questa sezione, esploriamo il circuito del kernel quantistico utilizzando i gate RZZ per introdurre il flusso di lavoro per i gate frazionari.

Iniziamo costruendo un circuito quantistico per calcolare le singole voci della matrice del kernel.
Questo viene fatto combinando circuiti di feature map ZZ con un overlap unitario.
La funzione kernel prende vettori nello spazio mappato delle feature e restituisce il loro prodotto interno come voce della matrice del kernel:
$$K(x, y) = \langle \Phi(x) | \Phi(y) \rangle,$$
dove $|\Phi(x)\rangle$ rappresenta lo stato quantistico mappato dalle feature.

Costruiamo manualmente un circuito di feature map ZZ utilizzando gate RZZ.
Sebbene Qiskit fornisca una `zz_feature_map` integrata, attualmente non supporta i gate RZZ a partire da Qiskit v2.0.2 ([vedere issue](https://github.com/Qiskit/qiskit/issues/14469)).

Successivamente, calcoliamo la funzione kernel per input identici - per esempio, $K(x, x) = 1$.
Sui computer quantistici rumorosi, questo valore potrebbe essere inferiore a 1 a causa del rumore.
Un risultato più vicino a 1 indica un rumore più basso nell'esecuzione.
In questo tutorial, ci riferiamo a questo valore come *fedeltà*, definita come
$$\text{fidelity} = K(x, x).$$

In [6]:
circuits[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/b3d6341a-0.avif" alt="Output of the previous code cell" />

In the standard Qiskit patterns workflow, parameter values are typically passed to the Sampler or Estimator primitive as part of a PUB.
However, when using a backend that supports fractional gates, these parameter values must be explicitly assigned to the quantum circuit prior to transpilation.

In [7]:
b_qc = [
    circ.assign_parameters(param) for circ, param in zip(circuits, params)
]
b_qc[0].draw("mpl", fold=-1)

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/6c9c1977-0.avif" alt="Output of the previous code cell" />

I circuiti del kernel quantistico e i corrispondenti valori dei parametri vengono generati per sistemi da 4 a 40 qubit, e le loro fedeltà vengono successivamente valutate.

In [8]:
backend_f = service.backend(name=backend_name, use_fractional_gates=True)
# pm_f includes `FoldRzzAngle` pass
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_f
)

In [9]:
t_qc_f = pm_f.run(b_qc)
print(t_qc_f[0].count_ops())
t_qc_f[0].draw("mpl", fold=-1)

OrderedDict([('rz', 35), ('rzz', 18), ('x', 13), ('rx', 9), ('measure', 4), ('barrier', 2)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/a18e5c70-1.avif" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/b3d6341a-0.avif)

Nel flusso di lavoro standard dei pattern Qiskit, i valori dei parametri vengono tipicamente passati alla primitiva Sampler o Estimator come parte di un PUB.
Tuttavia, quando si utilizza un backend che supporta i gate frazionari, questi valori dei parametri devono essere esplicitamente assegnati al circuito quantistico prima della transpilazione.

In [10]:
nnl_f = [qc.num_nonlocal_gates() for qc in t_qc_f]
depth_f = [qc.depth() for qc in t_qc_f]
duration_f = [
    qc.estimate_duration(backend_f.target, unit="u") for qc in t_qc_f
]

![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/6c9c1977-0.avif)

### Step 2: Ottimizzare il problema per l'esecuzione sull'hardware quantistico

Traspiliamo quindi il circuito utilizzando il pass manager seguendo il pattern Qiskit standard.
Fornendo un backend che supporta i gate frazionari a `generate_preset_pass_manager`, viene incluso automaticamente un pass specializzato chiamato `FoldRzzAngle`.
Questo pass modifica il circuito per rispettare i vincoli dell'angolo RZZ.
Di conseguenza, i gate RZZ con valori negativi nella figura precedente vengono trasformati in valori positivi, e vengono aggiunti alcuni gate X aggiuntivi.

In [11]:
sampler_f = SamplerV2(mode=backend_f)
sampler_f.options.dynamical_decoupling.enable = True
sampler_f.options.dynamical_decoupling.sequence_type = "XY4"
sampler_f.options.dynamical_decoupling.skip_reset_qubits = True

In [12]:
job = sampler_f.run(t_qc_f, shots=shots)
print(job.job_id())

d4bninsi51bc738j97eg


### Step 4: Post-process and return result in desired classical format

You can obtain the kernel function value $K(x, x)$ by measuring the probability of the all-zero bitstring `00...00` in the output.

In [13]:
# job = service.job("d1obougt0npc73flhiag")
result = job.result()
fidelity_f = [fidelity(result=res) for res in result]
print(fidelity_f)
usage_f = job.usage()

[0.9005, 0.647, 0.3345, 0.355, 0.3315, 0.174, 0.1875, 0.149, 0.1175, 0.085]


![Output of the previous code cell](../docs/images/tutorials/fractional-gates/extracted-outputs/a18e5c70-1.avif)

Per valutare l'impatto dei gate frazionari, valutiamo il numero di gate non locali (CZ e RZZ per questo backend),
insieme alle profondità e alle durate dei circuiti, e confrontiamo successivamente queste metriche con quelle di un flusso di lavoro standard.

In [14]:
# step 1: map classical inputs to quantum problem
# `circuits` and `params` from the previous section are reused here

In [15]:
# step 2: optimize circuits
backend_c = service.backend(backend_name)  # w/o fractional gates
pm_c = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_c
)
t_qc_c = pm_c.run(circuits)
print(t_qc_c[0].count_ops())
t_qc_c[0].draw("mpl", fold=-1)

OrderedDict([('rz', 130), ('sx', 80), ('cz', 36), ('measure', 4), ('barrier', 2)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/a10f2d95-1.avif" alt="Output of the previous code cell" />

In [16]:
nnl_c = [qc.num_nonlocal_gates() for qc in t_qc_c]
depth_c = [qc.depth() for qc in t_qc_c]
duration_c = [
    qc.estimate_duration(backend_c.target, unit="u") for qc in t_qc_c
]

In [17]:
# step 3: execute
sampler_c = SamplerV2(backend_c)
sampler_c.options.dynamical_decoupling.enable = True
sampler_c.options.dynamical_decoupling.sequence_type = "XY4"
sampler_c.options.dynamical_decoupling.skip_reset_qubits = True

In [18]:
job = sampler_c.run(pubs=zip(t_qc_c, params), shots=shots)
print(job.job_id())

d4bnirvnmdfs73ae3a2g


In [19]:
# step 4: post-processing
# job = service.job("d1obp8j3rr0s73bg4810")
result = job.result()
fidelity_c = [fidelity(res) for res in result]
print(fidelity_c)
usage_c = job.usage()

[0.6675, 0.5725, 0.098, 0.102, 0.065, 0.0235, 0.006, 0.0015, 0.0015, 0.002]


## Confronto del flusso di lavoro e del circuito senza gate frazionari
In questa sezione, presentiamo il flusso di lavoro standard dei pattern Qiskit utilizzando un backend che non supporta i gate frazionari.
Confrontando i circuiti transpilati, noterete che la versione che utilizza i gate frazionari (della sezione precedente) è più compatta di quella senza gate frazionari.

In [20]:
plt.plot(qubits, depth_c, "-o", label="no fractional gates")
plt.plot(qubits, depth_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("depth")
plt.title("Comparison of depths")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/ef343a53-1.avif" alt="Output of the previous code cell" />

In [21]:
plt.plot(qubits, duration_c, "-o", label="no fractional gates")
plt.plot(qubits, duration_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("duration (µs)")
plt.title("Comparison of durations")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/98bb2cd0-1.avif" alt="Output of the previous code cell" />

In [22]:
plt.plot(qubits, nnl_c, "-o", label="no fractional gates")
plt.plot(qubits, nnl_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("number of non-local gates")
plt.title("Comparison of numbers of non-local gates")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/1383b242-1.avif" alt="Output of the previous code cell" />

In [23]:
plt.plot(qubits, fidelity_c, "-o", label="no fractional gates")
plt.plot(qubits, fidelity_f, "-o", label="with fractional gates")
plt.xlabel("number of qubits")
plt.ylabel("fidelity")
plt.title("Comparison of fidelities")
plt.grid()
plt.legend()

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/8b4594f5-1.avif" alt="Output of the previous code cell" />

We compare the QPU usage time with and without fractional gates. The results in the following cell show that the QPU usage times are almost identical.

In [24]:
print(f"no fractional gates: {usage_c} seconds")
print(f"fractional gates: {usage_f} seconds")

no fractional gates: 7 seconds
fractional gates: 7 seconds


## Advanced topic: Using only fractional RX gates

The need for the modified workflow when using fractional gates primarily stems from the restriction on RZZ gate angles.
However, if you use only the fractional RX gates and exclude the fractional RZZ gates, you can continue to follow the standard Qiskit patterns workflow.
This approach can still offer meaningful benefits, particularly in circuits that involve a large number of RX gates and U gates, by reducing the overall gate count and potentially improving performance.
In this section, we demonstrate how to optimize your circuits using only fractional RX gates, while omitting RZZ gates.

To support this, we provide a utility function that allows you to disable a specific basis gate in a Target object.
Here, we use it to disable RZZ gates.

In [25]:
from qiskit.circuit.library import n_local
from qiskit.transpiler import Target

In [26]:
def remove_instruction_from_target(target: Target, gate_name: str) -> Target:
    new_target = Target(
        description=target.description,
        num_qubits=target.num_qubits,
        dt=target.dt,
        granularity=target.granularity,
        min_length=target.min_length,
        pulse_alignment=target.pulse_alignment,
        acquire_alignment=target.acquire_alignment,
        qubit_properties=target.qubit_properties,
        concurrent_measurements=target.concurrent_measurements,
    )

    for name, qarg_map in target.items():
        if name == gate_name:
            continue
        instruction = target.operation_from_name(name)
        if qarg_map == {None: None}:
            qarg_map = None
        new_target.add_instruction(instruction, qarg_map, name=name)
    return new_target

We use a circuit consisting of U, CZ, and RZZ gates as an example.

In [27]:
qc = n_local(3, "u", "cz", "linear", reps=1)
qc.rzz(1.1, 0, 1)
qc.draw("mpl")

<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/6b812497-0.avif" alt="Output of the previous code cell" />

We first transpile the circuit for a backend that does not support fractional gates.

In [28]:
pm_c = generate_preset_pass_manager(
    optimization_level=optimization_level, backend=backend_c
)
t_qc = pm_c.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 23), ('sx', 16), ('cz', 4)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/9e8e0709-1.avif" alt="Output of the previous code cell" />

## Confronto tra profondità e fedeltà
In questa sezione confrontiamo il numero di porte non locali e le fedeltà tra circuiti con e senza porte frazionarie.
Questo evidenzia i potenziali vantaggi dell'utilizzo delle porte frazionarie in termini di efficienza di esecuzione e qualità.

In [29]:
backend_f = service.backend(backend_name, use_fractional_gates=True)
target = remove_instruction_from_target(backend_f.target, "rzz")
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level,
    target=target,
)
t_qc = pm_f.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 22), ('sx', 14), ('cz', 4), ('rx', 1)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/db45feb0-1.avif" alt="Output of the previous code cell" />

### Optimize U gates with fractional RX gates

In this section, we demonstrate how to optimize U gates using fractional RX gates, building on the same circuit introduced in the previous section.

You will need to install the `qiskit-basis-constructor` [package](https://github.com/Qiskit/qiskit-basis-constructor) for this section.
This is a beta version of a new transpilation plugin for Qiskit, which might be integrated into Qiskit in the future.

In [30]:
# %pip install qiskit-basis-constructor

In [31]:
from qiskit.circuit.library import UGate
from qiskit_basis_constructor import DEFAULT_EQUIVALENCE_LIBRARY

We transpile the circuit using only fractional RX gates, excluding RZZ gates.
By introducing a custom decomposition rule, as shown in the following,
we can reduce the number of single-qubit gates required to implement a U gate.

This feature is currently under discussion in this [GitHub issue.](https://github.com/Qiskit/qiskit/issues/13455)

In [32]:
# special decomposition rule for UGate
x = ParameterVector("x", 3)
zxz = QuantumCircuit(1)
zxz.rz(x[2] - np.pi / 2, 0)
zxz.rx(x[0], 0)
zxz.rz(x[1] + np.pi / 2, 0)
DEFAULT_EQUIVALENCE_LIBRARY.add_equivalence(UGate(x[0], x[1], x[2]), zxz)

Next, we apply the transpiler using `constructor-beta` translation provided by `qiskit-basis-constructor` package.
As a result, the total number of gates is reduced compare to the previous transpilation.

In [33]:
pm_f = generate_preset_pass_manager(
    optimization_level=optimization_level,
    target=target,
    translation_method="constructor-beta",
)
t_qc = pm_f.run(qc)
print(t_qc.count_ops())
t_qc.draw("mpl")

OrderedDict([('rz', 16), ('rx', 9), ('cz', 4)])


<Image src="../docs/images/tutorials/fractional-gates/extracted-outputs/b19aae7c-1.avif" alt="Output of the previous code cell" />

## Troubleshooting

### Issue: Invalid RZZ angles might remain after transpilation

As of Qiskit v2.0.3, there are known issues where RZZ gates with invalid angles may remain in the circuits even after transpilation.
The issue typically arises under the following conditions.

#### Failure when using `target` option with `generate_preset_pass_manager` or `transpiler`

When the `target` option is used with `generate_preset_pass_manager` or `transpiler`, the specialized transpiler pass `FoldRzzAngle` is not invoked.
To ensure proper handling of RZZ angles for fractional gates, we recommend always using the `backend` option instead.
See [this issue](https://github.com/Qiskit/qiskit/issues/14318) for more details.

#### Failure when circuits contain certain gates

If your circuit includes certain gates such as `XXPlusYYGate`, the Qiskit transpiler may generate RZZ gates with invalid angles.
If you encounter this issue, see this [GitHub issue](https://github.com/Qiskit/qiskit-ibm-runtime/issues/2256#issuecomment-2889487152) for a workaround.

## Tutorial survey

Please take this short survey to provide feedback on this tutorial. Your insights will help us improve our content offerings and user experience.

[Link to survey](https://your.feedback.ibm.com/jfe/form/SV_cCNiGkGX5xZMzoG)